In [7]:
import sys
sys.path.append("../")
import io

from document_search.ocr import DocumentReader, EntityProcessor


In [2]:
documents = [
    "../data/SP-51.13330.2011.pdf",
    "../data/SP-60.13330.2020.pdf"
]


In [8]:
doc_reader = DocumentReader()

raw_documents = []
processed_docs = []
for document_path in documents:
    with open(document_path, 'rb') as f:
        raw_documents.append(io.BytesIO(f.read()))

    processed_doc, errors = doc_reader.read(document_path)
    print(document_path, errors)
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    processed_docs.append(processed_doc)


../data/SP-51.13330.2011.pdf []
../data/SP-60.13330.2020.pdf []


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/atom/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(
/home/atom/anaconda3/envs/doc-search2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
for doc, raw_doc in zip(processed_docs, raw_documents):
    storage.add_document(doc, raw_doc, pbar=True)


In [11]:
query = "здания, в которых на фасаде, обращенном в сторону внешнего источника шума, установлены шумозащитные окна"

storage.get_relevant_entities(query, 5)


[(TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011.pdf', page_number=6, document_id=None), text='здания, в которых на фасаде, обращенном в сторону внешнего источника шума, установлены шумозащитные окна, снабженные специальными вентиляционными устройствами с глушителями шума;'),
  0.96716064),
 (TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011.pdf', page_number=47, document_id=None), text='конструктивных  мер,  предусматривающих  строительство  придорожных  экранов,  установку шумозащитных окон в зданиях, расположенных в зонах неблагоприятного шумового воздействия;'),
  0.9018868),
 (TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011.pdf', page_number=9, document_id=None), text='ограждающих  конструкций,  обеспечивающих  требуемую  звукоизоляцию  от  внутренних  и внешних источников шума;'),
  0.8898582),
 (TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011.pdf', page_number=49, document_id=None), text='объе

In [12]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"

storage.get_relevant_entities(query, 5)


[(TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=66, document_id=None), text='10.6  Через  помещение  для  вентиляционного  оборудования  не  допускается  прокладывать трубопроводы:'),
  0.91254383),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=77, document_id=None), text='Не  допускается  прокладка  магистральных  и  разводящих  трубопроводов  систем  отопления  и внутреннего теплоснабжения через помещения жилых квартир, а также установка в них арматуры и спускных устройств общедомовых систем.'),
  0.8962804),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=50, document_id=None), text='7.10.14  Оборудование  вытяжных  систем  производственных,  административно-бытовых  и общественных  зданий,  удаляющих  воздух  с  резким  или  неприятным  запахом  (из  общественных уборных,  курительных  комнат  и  др.),  не  допускается  размещать  в  общем  помещении  для 